In [ ]:
# Thuc hien lay cac url tu cac bai viet tren trang CNN
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import os
import config

FILE_PATH = os.path.join(os.getcwd(), "cnn_links.txt")

def write_links_to_file(links, file_path):
    with open(file_path, 'a+', encoding='utf-8') as file:
        for link in links:
            if not link.startswith("http"):
                file.write(f"https://edition.cnn.com{link}\n")
                
                
def scrape_cnn_articles():
    service = Service(config.CHROME_DRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    categories = [
        '', 'health', 'us', 'world', 'politics', 'business', 'opinion', 
        'entertainment', 'style', 'travel', 'sports', 'climate', 'weather'
    ]
    links = []

    for category in categories:
        driver.get(f"https://edition.cnn.com/{category}")
        time.sleep(5)
        # Find all div elements with the attribute data-open-link
        elements = driver.find_elements(By.CSS_SELECTOR, "div[data-open-link]")
        # Extract the values of data-open-link and add them to the links list
        for element in elements:
            temp = element.get_attribute("data-open-link")
            if "video" not in temp:
                links.append(temp)

    write_links_to_file(links, FILE_PATH)

    # Close the browser
    driver.quit()

if __name__ == "__main__":
    scrape_cnn_articles()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import config

# Cấu hình ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Chạy Chrome ở chế độ headless
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Đường dẫn đến ChromeDriver
service = Service(config.CHROME_DRIVER_PATH)  # Thay thế đường dẫn đến chromedriver của bạn

# Tạo đối tượng trình duyệt
driver = webdriver.Chrome(service=service, options=chrome_options)

# Đọc từ file ra danh sách các urls
with open('cnn_links.txt', 'r') as f:
    urls = [line.strip() for line in f.readlines()]

print(f"Have {len(urls)} urls")

new_urls = urls.copy()
invalid_urls = []

try:
    for url in new_urls:
        try:
            driver.get(url)
            # Tìm tất cả các thẻ a có class phù hợp
            elements = driver.find_elements(By.CLASS_NAME, "card.container__item.container__item--type-media-image.container__item--type-.container_list-headlines-with-read-times__item.container_list-headlines-with-read-times__item--type-")
            # In ra giá trị của thuộc tính href cho mỗi thẻ a tìm được
            for element in elements:
                href = element.get_attribute('data-open-link')
                new_urls.append(href)
        except Exception as e:
            print(f"Error accessing {url}: {e}")
            invalid_urls.append(url)
except KeyboardInterrupt:
    print("Process interrupted by user.")
finally:
    print(f"Have new {len(new_urls)} URLs")
    with open('new_urls.txt', 'a+') as f:
        for url in new_urls:
            f.write(url + '\n')

    print(f"Have {len(invalid_urls)} invalid URLs")
    with open('invalid_urls.txt', 'a+') as f:
        for url in invalid_urls:
            f.write(url + '\n')

    driver.quit()


In [4]:
with open('new_urls.txt', 'r') as f:
    urls = f.readlines()
    _urls = set(urls)
    fixed_urls = list(_urls)
    with open('new_urls.txt', 'w') as f:
        for url in fixed_urls:
            f.write(url)